In [68]:
import streamlit as st
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings

In [69]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [90]:
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

In [71]:
llm = ChatOpenAI(model='gpt-4')

In [72]:
doc = WebBaseLoader("https://python.langchain.com/docs/get_started/quickstart")

In [73]:
docs = doc.load()

In [74]:
docs

[Document(page_content='\n\n\n\n\nQuickstart | 🦜️🔗 Langchain\n\n\n\n\n\n\n\nSkip to main content🦜️🔗 LangChainDocsUse casesIntegrationsGuidesAPIMoreVersioningChangelogDeveloper\'s guideTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS DocsChatSearchGet startedIntroductionInstallationQuickstartSecurityLangChain Expression LanguageGet startedWhy use LCELInterfaceStreamingHow toCookbookLangChain Expression Language (LCEL)ModulesModel I/ORetrievalAgentsChainsMoreLangServeLangSmithLangGraphGet startedQuickstartOn this pageQuickstartIn this quickstart we\'ll show you how to:Get setup with LangChain, LangSmith and LangServeUse the most basic and common components of LangChain: prompt templates, models, and output parsersUse LangChain Expression Language, the protocol that LangChain is built on and which facilitates component chainingBuild a simple application with LangChainTrace your application with LangSmithServe your 

In [75]:
text_split = RecursiveCharacterTextSplitter()

docs_chunk = text_split.split_documents(docs)

vector_store = Chroma.from_documents(docs_chunk, OpenAIEmbeddings())



In [76]:
vector_store

In [85]:
retriever = vector_store.as_retriever()

In [86]:
prompt = ChatPromptTemplate.from_messages(
    [SystemMessage(content="You are a helpful chatbot. Given the below conversation, generate a search query to look up in order to get information relevant to the conversation"),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}")]
)

In [87]:
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [84]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main content🦜️🔗 LangChainDocsUse casesIntegrationsGuidesAPIMoreVersioningChangelogDeveloper\'s guideTemplatesCookbooksTutorialsYouTube🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS DocsChatSearchGet startedIntroductionInstallationQuickstartSecurityLangChain Expression LanguageGet startedWhy use LCELInterfaceStreamingHow toCookbookLangChain Expression Language (LCEL)ModulesModel I/ORetrievalAgentsChainsMoreLangServeLangSmithLangGraphGet startedQuickstartOn this pageQuickstartIn this quickstart we\'ll show you how to:Get setup with LangChain, LangSmith and LangServeUse the most basic and common components of LangChain: prompt templates, models, and output parsersUse LangChain Expression Language, the protocol that LangChain is built on and which facilitates component chainingBuild a simple application with LangChainTrace your application with LangSmithServe your application with LangServeThat\'s a fair amount to c

In [88]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
      ("system", "Answer the user's questions based on the below context:\n\n{context}"),
      MessagesPlaceholder(variable_name="chat_history"),
      ("user", "{input}"),
    ]
)

In [92]:
stuff_documents_chain = create_stuff_documents_chain(llm,chat_prompt)

In [93]:
final_chain = create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [96]:
response = final_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [97]:
response['answer']

"LangSmith is designed to help you inspect and understand what's going on inside your chain or agent when building applications with LangChain. As these applications get more complex with multiple steps and multiple invocations of LLM (Large Language Model) calls, LangSmith allows you to trace your application, enabling you to monitor its operations. This can be especially valuable for testing and debugging purposes. You can set your environment variables to start logging traces with LangSmith, giving you insights into the performance and functionality of your LLM applications."